# NULL COMPLEX TETRAD FOR GENERAL METRICS IN THE CASE OF SLOWLY ROTATING BLACK HOLES

In this notebook, we propose an educated guess to obtain a valid tetrad in the context of entangled relativity for the case of slow rotation. We proceed step by step, starting from a tetrad valid for the Kerr metric, then gradually simplify it while ensuring that the tetrad's orthogonality conditions remain valid. Finally, we obtain a fairly general form for a non-diagonal metric, valid in the case of slow rotation.

The complexity of some calculations in this notebook prompted us to perform intermediate first order Taylor expansions to maintain tractability of the different operations using the Sagemath language. This does not affect the validity of our results since they are part of a first-order analysis in 'a'.

In [1]:
version()

'SageMath version 10.1, Release Date: 2023-08-20'

In [2]:
%display latex

'SageMath version used is 10.1, Release Date: 2023-08-20'

In [3]:
from sage.manifolds.operators import dalembertian
from sage.manifolds.operators import grad
import time
import pickle
comput_time0 = time.perf_counter()

# I. GR: Kerr-Newman solutions

We declare the Kerr-Newman spacetime as a 4-dimensional Lorentzian manifold M

In [4]:
M = Manifold(4, 'M', latex_name=r'\mathcal{M}', structure='Lorentzian')
print(M)

4-dimensional Lorentzian manifold M


In [5]:
XY.<t,r,th,ph> = M.chart(r"t r:(0,+oo) th:(0,pi):\theta ph:(0,2*pi):\phi")
print(XY)

Chart (M, (t, r, th, ph))


Let's declare the parameters of the spacetime as symbolic variables in real domain:

In [6]:
var('m a q r_m r_p', domain='real')

(m, a, q, r_m, r_p)

Notice that in this notebook the parameter $a$ is the angular momentum divided by the mass ($a=J/m$). We get the (yet undefined) spacetime metric:

In [7]:
g = M.metric()

In [8]:
rho2 = r^2 + (a*cos(th))^2
Delta = r^2 -2*m*r + a^2 + q^2

g[0,0] = -1 + (2*m*r-q^2)/rho2
g[0,3] = -a*sin(th)^2*(2*m*r-q^2)/rho2
g[1,1], g[2,2] = rho2/Delta, rho2
g[3,3] = (r^2 + a^2 + (2*m*r-q^2)*(a*sin(th))^2/rho2)*sin(th)^2
g[3,0] = -a*sin(th)^2*(2*m*r-q^2)/rho2

show(LatexExpr(r'g_{\mu\nu} = '), g[:])

g_{\mu\nu} =  [                                  -(q^2 - 2*m*r)/(a^2*cos(th)^2 + r^2) - 1                                                                          0                                                                          0                            (q^2 - 2*m*r)*a*sin(th)^2/(a^2*cos(th)^2 + r^2)]
[                                                                         0                            (a^2*cos(th)^2 + r^2)/(a^2 + q^2 - 2*m*r + r^2)                                                                          0                                                                          0]
[                                                                         0                                                                          0                                                        a^2*cos(th)^2 + r^2                                                                          0]
[                           (q^2 - 2*m*r)*a*sin(th)^2/(a^2*cos(th)^2 + r^2)                                                                          0                                                                          0 -((q^2 - 2*m*r)*a^2*sin(th)^2/(a^2*cos(th)^2 + r^2) - a^2 - r^2)*sin(th)^2]

# II. Special case of $q=0$ (Kerr Metric) and $a=\mathcal{O}(\epsilon)$ (slow rotation)

In [9]:
g.apply_map(lambda f:f.taylor(a,0,1).subs(q=0))
show(LatexExpr(r'g_{\mu\nu} = '), g[:])

g_{\mu\nu} =  [ (2*m*r - r^2)/r^2                  0                  0 -2*a*m*sin(th)^2/r]
[                 0 -r^2/(2*m*r - r^2)                  0                  0]
[                 0                  0                r^2                  0]
[-2*a*m*sin(th)^2/r                  0                  0      r^2*sin(th)^2]

## II. 1. Null complex tetrad for the Kerr metric

We first introduce a null complex tetrad for the Kerr metric, valid for any rotation regime.

In [10]:
L = M.vector_field(name='L')
N = M.vector_field(name='N')
Mr = M.vector_field(name='M_real')
Mbar = M.vector_field(name='M_bar')

In [11]:
rho2 = r^2 + (a*cos(th))^2
Delta = r^2 -2*m*r + a^2
sigma = 1/(2**(1/2))*(1/(r+I*a*cos(th)))

In [12]:
L[0],N[0],Mr[0],Mbar[0] = (r**2+a**2)/Delta, (r**2+a**2)/(2*rho2), sigma*I*sin(th)*a, (sigma*I*sin(th)*a).conjugate()
L[1],N[1],Mr[1],Mbar[1] = 1, -Delta/(2*rho2), 0, 0
L[2],N[2],Mr[2],Mbar[2] = 0,0,sigma, sigma.conjugate()
L[3],N[3],Mr[3],Mbar[3] = a/Delta, a/(2*rho2), sigma*I/sin(th), (sigma*I/sin(th)).conjugate()

In [13]:
Ld, Md, Nd, Mbar_d= L.down(g), Mr.down(g), N.down(g), Mbar.down(g)
show(LatexExpr(r'l^{\mu} = '),L[:])
show(LatexExpr(r'n^{\mu} = '),N[:])
show(LatexExpr(r'm^{\mu} = '),Mr[:])
show(LatexExpr(r'\bar{m}^{\mu} = '), Mbar[:])

l^{\mu} =  [(a^2 + r^2)/(a^2 - 2*m*r + r^2), 1, 0, a/(a^2 - 2*m*r + r^2)]

n^{\mu} =  [1/2*(a^2 + r^2)/(a^2*cos(th)^2 + r^2),
 -1/2*(a^2 - 2*m*r + r^2)/(a^2*cos(th)^2 + r^2),
 0,
 1/2*a/(a^2*cos(th)^2 + r^2)]

m^{\mu} =  [1/2*I*sqrt(2)*a*sin(th)/(I*a*cos(th) + r),
 0,
 1/2*sqrt(2)/(I*a*cos(th) + r),
 1/2*I*sqrt(2)/((I*a*cos(th) + r)*sin(th))]

\bar{m}^{\mu} =  [-1/2*I*sqrt(2)*a*sin(th)/(-I*a*cos(th) + r),
 0,
 1/2*sqrt(2)/(-I*a*cos(th) + r),
 -1/2*I*sqrt(2)/((-I*a*cos(th) + r)*sin(th))]

Let's show that this tetrad is still valid for the Kerr metric in the case of a slow rotation (i.e. at the first order in 'a').

In [14]:
l_scalar, n_scalar, m_scalar, mbar_scalar = L['^{a}']*Ld['_{a}'], N['^{a}']*Nd['_{a}'], Mr['^{a}']*Md['_{a}'],Mbar['^{a}']*Mbar_d['_{a}']
lm, mn,ln, mbarm = L['^{a}']*Md['_{a}'], Mr['^{a}']*Nd['_{a}'], L['^{a}']*Nd['_{a}'], Mr['^{a}']*Mbar_d['_{a}']

show(LatexExpr(r'l^{\mu}l_{\mu} = '),l_scalar.expr().taylor(a,0,1).subs(q=0), LatexExpr(r'\hspace{0.5cm} n^{\mu}n_{\mu} = '),\
     n_scalar.expr().taylor(a,0,1).subs(q=0), LatexExpr(r'\hspace{0.5cm} m^{\mu}m_{\mu} = '),m_scalar.expr().taylor(a,0,1), \
     LatexExpr(r'\hspace{0.5cm} \bar{m}^{\mu}\bar{m}_{\mu} = '),mbar_scalar.expr().taylor(a,0,1))

show(LatexExpr(r'l^{\mu}m_{\mu} = '),lm.expr().taylor(a,0,1), LatexExpr(r'\hspace{0.5cm} m^{\mu}n_{\mu} = '),\
     mn.expr().taylor(a,0,1))


show(LatexExpr(r'l^{\mu}n_{\mu} = '),ln.expr().taylor(a,0,1).subs(q=0), LatexExpr(r'\hspace{0.5cm} m^{\mu}\bar{m}_{\mu} = '),mbarm.expr().taylor(a,0,1))

l^{\mu}l_{\mu} =  0 \hspace{0.5cm} n^{\mu}n_{\mu} =  0 \hspace{0.5cm} m^{\mu}m_{\mu} =  0 \hspace{0.5cm} \bar{m}^{\mu}\bar{m}_{\mu} =  0

l^{\mu}m_{\mu} =  0 \hspace{0.5cm} m^{\mu}n_{\mu} =  0

l^{\mu}n_{\mu} =  -1 \hspace{0.5cm} m^{\mu}\bar{m}_{\mu} =  1

We now check that we can reconstruct the metric from the components of the tetrad:

In [15]:
for i in range(4):
    g[i,i] = -2*Ld[i]*Nd[i] + 2*Md[i]*Mbar_d[i]
g[0,3] = -Ld[0]*Nd[3] - Ld[3]*Nd[0] + Md[0]*Mbar_d[3] + Md[3]*Mbar_d[0]
g.apply_map(lambda f:f.taylor(a,0,1))
show(LatexExpr(r'g_{\mu\nu} = '), g[:])

g_{\mu\nu} =  [       (2*m - r)/r                  0                  0 -2*a*m*sin(th)^2/r]
[                 0       -r/(2*m - r)                  0                  0]
[                 0                  0                r^2                  0]
[-2*a*m*sin(th)^2/r                  0                  0      r^2*sin(th)^2]

## II. 2. First simplifications of the tetrad form

Let's remove any term in "a" with an order higher than 1 and rescale the "m" component of the tetrad in order to preserve the equalities.

In [16]:
rho2 = r^2
Delta = r^2 -2*m*r
sigma = 1/(2**(1/2))/r

In [17]:
L[0],N[0],Mr[0],Mbar[0] = (r**2)/Delta, (r**2)/(2*rho2), sigma*I*sin(th)*a, (sigma*I*sin(th)*a).conjugate()
L[1],N[1],Mr[1],Mbar[1] = 1, -Delta/(2*rho2), 0, 0
L[2],N[2],Mr[2],Mbar[2] = 0,0,sigma, sigma.conjugate()
L[3],N[3],Mr[3],Mbar[3] = a/Delta, a/(2*rho2), sigma*I/sin(th), (sigma*I/sin(th)).conjugate()

In [18]:
Ld, Md, Nd, Mbar_d= L.down(g), Mr.down(g), N.down(g), Mbar.down(g)
show(LatexExpr(r'l^{\mu} = '),L[:])
show(LatexExpr(r'n^{\mu} = '),N[:])
show(LatexExpr(r'm^{\mu} = '),Mr[:])
show(LatexExpr(r'\bar{m}^{\mu} = '), Mbar[:])

l^{\mu} =  [-r^2/(2*m*r - r^2), 1, 0, -a/(2*m*r - r^2)]

n^{\mu} =  [1/2, 1/2*(2*m*r - r^2)/r^2, 0, 1/2*a/r^2]

m^{\mu} =  [1/2*I*sqrt(2)*a*sin(th)/r, 0, 1/2*sqrt(2)/r, 1/2*I*sqrt(2)/(r*sin(th))]

\bar{m}^{\mu} =  [-1/2*I*sqrt(2)*a*sin(th)/r, 0, 1/2*sqrt(2)/r, -1/2*I*sqrt(2)/(r*sin(th))]

In [19]:
l_scalar, n_scalar, m_scalar, mbar_scalar = L['^{a}']*Ld['_{a}'], N['^{a}']*Nd['_{a}'], Mr['^{a}']*Md['_{a}'],Mbar['^{a}']*Mbar_d['_{a}']
lm, mn,ln, mbarm = L['^{a}']*Md['_{a}'], Mr['^{a}']*Nd['_{a}'], L['^{a}']*Nd['_{a}'], Mr['^{a}']*Mbar_d['_{a}']

show(LatexExpr(r'l^{\mu}l_{\mu} = '),l_scalar.expr().taylor(a,0,1).subs(q=0), LatexExpr(r'\hspace{0.5cm} n^{\mu}n_{\mu} = '),\
     n_scalar.expr().taylor(a,0,1).subs(q=0), LatexExpr(r'\hspace{0.5cm} m^{\mu}m_{\mu} = '),m_scalar.expr().taylor(a,0,1), \
     LatexExpr(r'\hspace{0.5cm} \bar{m}^{\mu}\bar{m}_{\mu} = '),mbar_scalar.expr().taylor(a,0,1))

show(LatexExpr(r'l^{\mu}m_{\mu} = '),lm.expr().taylor(a,0,1), LatexExpr(r'\hspace{0.5cm} m^{\mu}n_{\mu} = '),\
     mn.expr().taylor(a,0,1))


show(LatexExpr(r'l^{\mu}n_{\mu} = '),ln.expr().taylor(a,0,1).subs(q=0), LatexExpr(r'\hspace{0.5cm} m^{\mu}\bar{m}_{\mu} = '),mbarm.expr().taylor(a,0,1))

l^{\mu}l_{\mu} =  0 \hspace{0.5cm} n^{\mu}n_{\mu} =  0 \hspace{0.5cm} m^{\mu}m_{\mu} =  0 \hspace{0.5cm} \bar{m}^{\mu}\bar{m}_{\mu} =  0

l^{\mu}m_{\mu} =  0 \hspace{0.5cm} m^{\mu}n_{\mu} =  0

l^{\mu}n_{\mu} =  -1 \hspace{0.5cm} m^{\mu}\bar{m}_{\mu} =  1

In [20]:
for i in range(4):
    g[i,i] = -2*Ld[i]*Nd[i] + 2*Md[i]*Mbar_d[i]
g[0,3] = -Ld[0]*Nd[3] - Ld[3]*Nd[0] + Md[0]*Mbar_d[3] + Md[3]*Mbar_d[0]
g.apply_map(lambda f:f.taylor(a,0,1).factor())
show(LatexExpr(r'g_{\mu\nu} = '), g[:])

g_{\mu\nu} =  [       (2*m - r)/r                  0                  0 -2*a*m*sin(th)^2/r]
[                 0       -r/(2*m - r)                  0                  0]
[                 0                  0                r^2                  0]
[-2*a*m*sin(th)^2/r                  0                  0      r^2*sin(th)^2]

The tetrad conditions are still verified

# III. Generalization

The quantities \(m\), \(a\) and \(r\) do not have a fixed role a priori and can, from a mathematical point of view, be replaced by any arbitrary function. We first replace \(r\) by a function $\epsilon(r)$, then we find the expression of \(m\) to make appear the function $\sigma(r)$ and finally  \(a\) is chosen such that we get $f(r)$. This way, we can obtain a more general form of the metric.

In [21]:
g = M.metric()
sig = function('sigma')
eps = function('epsilon')
f = function('f')
g[0,0] = sig(r)
g[0,3] = -a*f(r)*sin(th)**2 
g[1,1] = -1/sig(r)
g[2,2] = eps(r)
g[3,3] = sin(th)**2*eps(r)
g[3,0] = -a*f(r)*sin(th)**2  
g[3,0] = -a*f(r)*sin(th)**2 

In [22]:
show(LatexExpr(r'g_{\mu\nu} = '), g[:])

g_{\mu\nu} =  [            sigma(r)                    0                    0    -a*f(r)*sin(th)^2]
[                   0          -1/sigma(r)                    0                    0]
[                   0                    0           epsilon(r)                    0]
[   -a*f(r)*sin(th)^2                    0                    0 epsilon(r)*sin(th)^2]

In [23]:
Delta = a*f(r)/((sig(r)+1)*eps(r))

In [24]:
L[0],N[0],Mr[0],Mbar[0] = -1/sig(r), 1/2, Delta*eps(r)**(1/2)*I*sin(th)/(2**(1/2)), -Delta*eps(r)**(1/2)*I*sin(th)/(2**(1/2))
L[1],N[1],Mr[1],Mbar[1] = 1, sig(r)/2, 0, 0
L[2],N[2],Mr[2],Mbar[2] = 0,0,1/((2*eps(r))**(1/2)), 1/((2*eps(r))**(1/2))
L[3],N[3],Mr[3],Mbar[3] = -Delta/sig(r), Delta/2, I/(2**(1/2)*eps(r)**(1/2)*sin(th)), -I/(2**(1/2)*eps(r)**(1/2)*sin(th))
Ld, Md, Nd, Mbar_d= L.down(g), Mr.down(g), N.down(g), Mbar.down(g)


show(LatexExpr(r'l^{\mu} = '),L[:])
show(LatexExpr(r'n^{\mu} = '),N[:])
show(LatexExpr(r'm^{\mu} = '),Mr[:])
show(LatexExpr(r'\bar{m}^{\mu} = '), Mbar[:])

l^{\mu} =  [-1/sigma(r), 1, 0, -a*f(r)/((sigma(r) + 1)*epsilon(r)*sigma(r))]

n^{\mu} =  [1/2, 1/2*sigma(r), 0, 1/2*a*f(r)/((sigma(r) + 1)*epsilon(r))]

m^{\mu} =  [1/2*I*sqrt(2)*a*f(r)*sin(th)/((sigma(r) + 1)*sqrt(epsilon(r))),
 0,
 1/2*sqrt(2)/sqrt(epsilon(r)),
 1/2*I*sqrt(2)/(sqrt(epsilon(r))*sin(th))]

\bar{m}^{\mu} =  [-1/2*I*sqrt(2)*a*f(r)*sin(th)/((sigma(r) + 1)*sqrt(epsilon(r))),
 0,
 1/2*sqrt(2)/sqrt(epsilon(r)),
 -1/2*I*sqrt(2)/(sqrt(epsilon(r))*sin(th))]

In [25]:
l_scalar, n_scalar, m_scalar, mbar_scalar = L['^{a}']*Ld['_{a}'], N['^{a}']*Nd['_{a}'], Mr['^{a}']*Md['_{a}'],Mbar['^{a}']*Mbar_d['_{a}']
lm, mn,ln, mbarm = L['^{a}']*Md['_{a}'], Mr['^{a}']*Nd['_{a}'], L['^{a}']*Nd['_{a}'], Mr['^{a}']*Mbar_d['_{a}']

show(LatexExpr(r'l^{\mu}l_{\mu} = '),l_scalar.expr().taylor(a,0,1), LatexExpr(r'\hspace{0.5cm} n^{\mu}n_{\mu} = '),\
     n_scalar.expr().taylor(a,0,1).subs(q=0), LatexExpr(r'\hspace{0.5cm} m^{\mu}m_{\mu} = '),m_scalar.expr().taylor(a,0,1), \
     LatexExpr(r'\hspace{0.5cm} \bar{m}^{\mu}\bar{m}_{\mu} = '),mbar_scalar.expr().taylor(a,0,1))

show(LatexExpr(r'l^{\mu}m_{\mu} = '),lm.expr().taylor(a,0,1), LatexExpr(r'\hspace{0.5cm} m^{\mu}n_{\mu} = '),\
     mn.expr().taylor(a,0,1))


show(LatexExpr(r'l^{\mu}n_{\mu} = '),ln.expr().taylor(a,0,1), LatexExpr(r'\hspace{0.5cm} m^{\mu}\bar{m}_{\mu} = '),mbarm.expr().taylor(a,0,1))

l^{\mu}l_{\mu} =  0 \hspace{0.5cm} n^{\mu}n_{\mu} =  0 \hspace{0.5cm} m^{\mu}m_{\mu} =  0 \hspace{0.5cm} \bar{m}^{\mu}\bar{m}_{\mu} =  0

l^{\mu}m_{\mu} =  0 \hspace{0.5cm} m^{\mu}n_{\mu} =  0

l^{\mu}n_{\mu} =  -1 \hspace{0.5cm} m^{\mu}\bar{m}_{\mu} =  1

In [26]:
for i in range(4):
    g[i,i] = -2*Ld[i]*Nd[i] + 2*Md[i]*Mbar_d[i]
g[0,3] = -Ld[0]*Nd[3] - Ld[3]*Nd[0] + Md[0]*Mbar_d[3] + Md[3]*Mbar_d[0]
g.apply_map(lambda f:f.taylor(a,0,1))
show(LatexExpr(r'g_{\mu\nu} = '), g[:])

g_{\mu\nu} =  [            sigma(r)                    0                    0    -a*f(r)*sin(th)^2]
[                   0          -1/sigma(r)                    0                    0]
[                   0                    0           epsilon(r)                    0]
[   -a*f(r)*sin(th)^2                    0                    0 epsilon(r)*sin(th)^2]

Thus, we have obtained a form of tetrad valid at the first order in 'a' for a non-diagonal metric composed of functions $\epsilon(r)$, $\sigma(r)$ and $f(r)$

## III.1. Conformal transformation

It is important to note that the solutions of entangled relativity, for which we wish to derive a null tetrad, do not exactly follow the general form presented above as they are defined as its conformal transformation. Since the properties of orthogonality and nullity are invariant under conformal transformation, the tetrad constructed here remains valid for our solutions, although their normalization is now rescaled. Consequently, the algebraic classification of the spacetime also remains invariant.

# IV. Case of slowly rotating black holes in Entangled Relativity 

In [27]:
O = function('O')
Psi(r) = (1-r_p/r)
Zeta(r) = (1-r_m/r)

A(r)=-Psi(r)*Zeta(r)**(11/13)
C(r)=r**2*Zeta(r)**(2/13)
E(r)=(r**2*(1+(1/(2*(3)**(1/2)))**2)**2)*Zeta(r)**(2*(1/(2*(3)**(1/2)))**2/(1+(1/(2*(3)**(1/2)))**2))/(((1-(1/(2*(3)**(1/2)))**2)*(1-3*(1/(2*(3)**(1/2)))**2)*r_m**2))-Zeta(r)**((1-(1/(2*(3)**(1/2)))**2)/(1+(1/(2*(3)**(1/2)))**2))*(1+(1+(1/(2*(3)**(1/2)))**2)**2*r**2/((1-(1/(2*(3)**(1/2)))**2)*(1-3*(1/(2*(3)**(1/2)))**2)*r_m**2)+(1+(1/(2*(3)**(1/2)))**2)*r/((1-(1/(2*(3)**(1/2)))**2)*r_m)-r_p/r)

In [28]:
elec = False

In [29]:
if elec:
    Omega(r)=Zeta(r)**(2/13)
else:
    Omega(r)=Zeta(r)**(-2/13)

In [30]:
k = g.copy()
k.set_name('k')
k.apply_map(lambda l: l*O(r)**2) 
k.apply_map(lambda l: l.substitute_function(O, Omega).substitute_function(sig, A).substitute_function(eps, C).substitute_function(f, E)) 
k.display()

k = (-r_m/r + 1)^(7/13)*(r_p/r - 1) dt⊗dt + 1/99*((169*r^2/r_m^2 + 117*r/r_m - 99*r_p/r + 99)*(-r_m/r + 1)^(11/13) - 169*r^2*(-r_m/r + 1)^(2/13)/r_m^2)*a*sin(th)^2/(-r_m/r + 1)^(4/13) dt⊗dph - 1/((-r_m/r + 1)^(15/13)*(r_p/r - 1)) dr⊗dr + r^2/(-r_m/r + 1)^(2/13) dth⊗dth + 1/99*((169*r^2/r_m^2 + 117*r/r_m - 99*r_p/r + 99)*(-r_m/r + 1)^(11/13) - 169*r^2*(-r_m/r + 1)^(2/13)/r_m^2)*a*sin(th)^2/(-r_m/r + 1)^(4/13) dph⊗dt + r^2*sin(th)^2/(-r_m/r + 1)^(2/13) dph⊗dph

In [31]:
for j in [L, N, Mr, Mbar]:
    j.apply_map(lambda l: l.substitute_function(O, Omega).substitute_function(sig, A).substitute_function(eps, C).substitute_function(f, E)) 

show(LatexExpr(r'l^{\mu} = '),L[:])
show(LatexExpr(r'n^{\mu} = '),N[:])
show(LatexExpr(r'm^{\mu} = '),Mr[:])
show(LatexExpr(r'\bar{m}^{\mu} = '), Mbar[:])

l^{\mu} =  [-1/((-r_m/r + 1)^(11/13)*(r_p/r - 1)),
 1,
 0,
 -1/99*((169*r^2/r_m^2 + 117*r/r_m - 99*r_p/r + 99)*(-r_m/r + 1)^(11/13) - 169*r^2*(-r_m/r + 1)^(2/13)/r_m^2)*a/(((-r_m/r + 1)^(11/13)*(r_p/r - 1) + 1)*r^2*(r_m/r - 1)*(r_p/r - 1))]

n^{\mu} =  [1/2,
 1/2*(-r_m/r + 1)^(11/13)*(r_p/r - 1),
 0,
 -1/198*((169*r^2/r_m^2 + 117*r/r_m - 99*r_p/r + 99)*(-r_m/r + 1)^(11/13) - 169*r^2*(-r_m/r + 1)^(2/13)/r_m^2)*a/(((-r_m/r + 1)^(11/13)*(r_p/r - 1) + 1)*r^2*(-r_m/r + 1)^(2/13))]

m^{\mu} =  [-1/198*I*sqrt(2)*((169*r^2/r_m^2 + 117*r/r_m - 99*r_p/r + 99)*(-r_m/r + 1)^(11/13) - 169*r^2*(-r_m/r + 1)^(2/13)/r_m^2)*a*sin(th)/(((-r_m/r + 1)^(11/13)*(r_p/r - 1) + 1)*r*(-r_m/r + 1)^(1/13)),
 0,
 1/2*sqrt(2)/(r*(-r_m/r + 1)^(1/13)),
 1/2*I*sqrt(2)/(r*(-r_m/r + 1)^(1/13)*sin(th))]

\bar{m}^{\mu} =  [1/198*I*sqrt(2)*((169*r^2/r_m^2 + 117*r/r_m - 99*r_p/r + 99)*(-r_m/r + 1)^(11/13) - 169*r^2*(-r_m/r + 1)^(2/13)/r_m^2)*a*sin(th)/(((-r_m/r + 1)^(11/13)*(r_p/r - 1) + 1)*r*(-r_m/r + 1)^(1/13)),
 0,
 1/2*sqrt(2)/(r*(-r_m/r + 1)^(1/13)),
 -1/2*I*sqrt(2)/(r*(-r_m/r + 1)^(1/13)*sin(th))]

Here we have the final expression for the different tetrad components

In [32]:
expressions = {
    'L_expression': L,
    'N_expression': N,
    'M_expression': Mr,
    'Mbar_expression': Mbar,
}

with open('SR_electric_tetrad_.pkl' if elec else 'SR_magnetic_tetrad_.pkl', 'wb') as file:
    pickle.dump(expressions, file)


# Appendix

## A.1. Conformal Transformations

In this section, we detail how the null complex is affected by conformal transformation, in particular how orthogonality and nullity are conserved, while the normalization is now rescaled.

In [33]:
p = g.copy()
p.set_name('p')
O = function('O')

In [34]:
L[0],N[0],Mr[0],Mbar[0] = -1/sig(r), 1/2, Delta*eps(r)**(1/2)*I*sin(th)/(2**(1/2)), -Delta*eps(r)**(1/2)*I*sin(th)/(2**(1/2))
L[1],N[1],Mr[1],Mbar[1] = 1, sig(r)/2, 0, 0
L[2],N[2],Mr[2],Mbar[2] = 0,0,1/((2*eps(r))**(1/2)), 1/((2*eps(r))**(1/2))
L[3],N[3],Mr[3],Mbar[3] = -Delta/sig(r), Delta/2, I/(2**(1/2)*eps(r)**(1/2)*sin(th)), -I/(2**(1/2)*eps(r)**(1/2)*sin(th))
Ld, Md, Nd, Mbar_d= L.down(g), Mr.down(g), N.down(g), Mbar.down(g)


show(LatexExpr(r'l^{\mu} = '),L[:])
show(LatexExpr(r'n^{\mu} = '),N[:])
show(LatexExpr(r'm^{\mu} = '),Mr[:])
show(LatexExpr(r'\bar{m}^{\mu} = '), Mbar[:])

l^{\mu} =  [-1/sigma(r), 1, 0, -a*f(r)/((sigma(r) + 1)*epsilon(r)*sigma(r))]

n^{\mu} =  [1/2, 1/2*sigma(r), 0, 1/2*a*f(r)/((sigma(r) + 1)*epsilon(r))]

m^{\mu} =  [1/2*I*sqrt(2)*a*f(r)*sin(th)/((sigma(r) + 1)*sqrt(epsilon(r))),
 0,
 1/2*sqrt(2)/sqrt(epsilon(r)),
 1/2*I*sqrt(2)/(sqrt(epsilon(r))*sin(th))]

\bar{m}^{\mu} =  [-1/2*I*sqrt(2)*a*f(r)*sin(th)/((sigma(r) + 1)*sqrt(epsilon(r))),
 0,
 1/2*sqrt(2)/sqrt(epsilon(r)),
 -1/2*I*sqrt(2)/(sqrt(epsilon(r))*sin(th))]

In [35]:
p.apply_map(lambda f: f*O(r)**2) ; p[:]

[            O(r)^2*sigma(r)                           0                           0    -a*O(r)^2*f(r)*sin(th)^2]
[                          0            -O(r)^2/sigma(r)                           0                           0]
[                          0                           0           O(r)^2*epsilon(r)                           0]
[   -a*O(r)^2*f(r)*sin(th)^2                           0                           0 O(r)^2*epsilon(r)*sin(th)^2]

In [36]:
for n, v in enumerate([L, N, Mr, Mbar]):
    v.apply_map(lambda f: f*O(r)) 
    show(LatexExpr(r'l^{\mu} = ' if n == 0 else (r'n^{\mu} = ' if n==1 else (r'm^{\mu} = '\
                                        if n==2 else r'\bar{m}^{\mu} = '))),v[:])
Ld, Md, Nd, Mbar_d= L.down(g), Mr.down(g), N.down(g), Mbar.down(g)

l^{\mu} =  [-O(r)/sigma(r), O(r), 0, -a*O(r)*f(r)/((sigma(r) + 1)*epsilon(r)*sigma(r))]

n^{\mu} =  [1/2*O(r), 1/2*O(r)*sigma(r), 0, 1/2*a*O(r)*f(r)/((sigma(r) + 1)*epsilon(r))]

m^{\mu} =  [1/2*I*sqrt(2)*a*O(r)*f(r)*sin(th)/((sigma(r) + 1)*sqrt(epsilon(r))),
 0,
 1/2*sqrt(2)*O(r)/sqrt(epsilon(r)),
 1/2*I*sqrt(2)*O(r)/(sqrt(epsilon(r))*sin(th))]

\bar{m}^{\mu} =  [-1/2*I*sqrt(2)*a*O(r)*f(r)*sin(th)/((sigma(r) + 1)*sqrt(epsilon(r))),
 0,
 1/2*sqrt(2)*O(r)/sqrt(epsilon(r)),
 -1/2*I*sqrt(2)*O(r)/(sqrt(epsilon(r))*sin(th))]

In [37]:
l_scalar, n_scalar, m_scalar, mbar_scalar = L['^{a}']*Ld['_{a}'], N['^{a}']*Nd['_{a}'], Mr['^{a}']*Md['_{a}'],Mbar['^{a}']*Mbar_d['_{a}']
lm, mn,ln, mbarm = L['^{a}']*Md['_{a}'], Mr['^{a}']*Nd['_{a}'], L['^{a}']*Nd['_{a}'], Mr['^{a}']*Mbar_d['_{a}']

show(LatexExpr(r'l^{\mu}l_{\mu} = '),l_scalar.expr().taylor(a,0,1), LatexExpr(r'\hspace{0.5cm} n^{\mu}n_{\mu} = '),\
     n_scalar.expr().taylor(a,0,1).subs(q=0), LatexExpr(r'\hspace{0.5cm} m^{\mu}m_{\mu} = '),m_scalar.expr().taylor(a,0,1), \
     LatexExpr(r'\hspace{0.5cm} \bar{m}^{\mu}\bar{m}_{\mu} = '),mbar_scalar.expr().taylor(a,0,1))

show(LatexExpr(r'l^{\mu}m_{\mu} = '),lm.expr().taylor(a,0,1), LatexExpr(r'\hspace{0.5cm} m^{\mu}n_{\mu} = '),\
     mn.expr().taylor(a,0,1))


show(LatexExpr(r'l^{\mu}n_{\mu} = '),ln.expr().taylor(a,0,1), LatexExpr(r'\hspace{0.5cm} m^{\mu}\bar{m}_{\mu} = '),mbarm.expr().taylor(a,0,1))

l^{\mu}l_{\mu} =  0 \hspace{0.5cm} n^{\mu}n_{\mu} =  0 \hspace{0.5cm} m^{\mu}m_{\mu} =  0 \hspace{0.5cm} \bar{m}^{\mu}\bar{m}_{\mu} =  0

l^{\mu}m_{\mu} =  0 \hspace{0.5cm} m^{\mu}n_{\mu} =  0

l^{\mu}n_{\mu} =  -O(r)^2 \hspace{0.5cm} m^{\mu}\bar{m}_{\mu} =  O(r)^2

In [38]:
for i in range(4):
    k[i,i] = -2*Ld[i]*Nd[i] + 2*Md[i]*Mbar_d[i]
k[0,3] = -Ld[0]*Nd[3] - Ld[3]*Nd[0] + Md[0]*Mbar_d[3] + Md[3]*Mbar_d[0]
k.apply_map(lambda f:f.taylor(a,0,1))
show(LatexExpr(r'g_{\mu\nu} = '), k[:])

g_{\mu\nu} =  [            O(r)^2*sigma(r)                           0                           0    -a*O(r)^2*f(r)*sin(th)^2]
[                          0            -O(r)^2/sigma(r)                           0                           0]
[                          0                           0           O(r)^2*epsilon(r)                           0]
[   -a*O(r)^2*f(r)*sin(th)^2                           0                           0 O(r)^2*epsilon(r)*sin(th)^2]

## In conclusion we derived a valid null complex tetrad for both the electric and magnetic solutions in case of a slowly rotating black hole.
